<a href="https://colab.research.google.com/github/mikexcohen/Calculus_book/blob/main/ch09_applications_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculus unraveled: Intuition, Proofs, and Python**
### Mike X Cohen (sincxpress.com)
#### https://github.com/mikexcohen/calculus_book
#### Code for Chapter 9 (Applications of differentiation)

---

# About this code file:

### This notebook contains full code solutions to the exercises in this book chapter. There are many correct ways to solve the exercises; this notebook provides *a* solution, not *THE* solution. Please use this code as a starting point to continue exploring and experimenting with calculus concepts and visualizations.

## **Using the code without the book may lead to confusion or errors.**

#### This code was written in google-colab. The notebook may require some modifications if you use a different IDE.

In [ ]:
# import libraries and define global settings
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

# define global figure properties used for publication
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg') # display figures in vector format
plt.rcParams.update({'font.size':14,             # font size
                     'savefig.dpi':300,          # output resolution
                     'axes.titlelocation':'left',# title location
                     'axes.spines.right':False,  # remove axis bounding box
                     'axes.spines.top':False,    # remove axis bounding box
                     'lines.linewidth':2         # increase default line thickness
                     })

# Exercise 9.1: Approximation function

In [ ]:
# function that takes f, df, xa, xs, and returns estimate at xa
def linearApproximation(f,df,xa,xs):
  #   y  =        m      x   +    b
  return df.subs(x,xs)*(xa-xs) + f.subs(x,xs)

In [ ]:
x = sym.symbols('x')

# define the function in sympy
fx = sym.sqrt(x) + 2*x

# compute its derivative
df = sym.diff(fx,x)


xa = 4.37 # value to estimate
xs = 4 # convenient point

linearApproxValue = linearApproximation(fx,df,xa,xs)
trueValue = fx.subs(x,xa)

print(f'Linear approximation: {linearApproxValue}')
print(f'Sympy calculation   : {trueValue}')

# Exercise 9.2: Different starting values

In [ ]:
# list of starting values
startVals = np.linspace(5,14,30)
xa = 10
trueValueXa = fx.subs(x,xa)

# initializations
linapprox = np.zeros(len(startVals))
approxErrors = np.zeros(len(startVals))

# do the approximations and store the results
for i,xs in enumerate(startVals):
  linapprox[i] = linearApproximation(fx,df,xa,xs)
  approxErrors[i] = trueValueXa - linapprox[i]

# plot the results
plt.figure(figsize=(8,4))
plt.plot(startVals,approxErrors,'ks',markerfacecolor='w',markersize=9)
plt.axhline(0,linestyle='--',color=[.7,.7,.7],zorder=-3)
plt.axvline(xa,linestyle=':',color=[.8,.8,.8],zorder=-3)
plt.xlabel(r'Initial guess ($x_s$)')
plt.ylabel('Error from "true" value')

plt.tight_layout()
plt.savefig('diffApps_ex2.png')
plt.show()

# Exercise 9.3: A different function

In [ ]:
x = sym.symbols('x')

# define the function in sympy
fx = sym.Abs( sym.sin(x) )

# compute its derivative
df = sym.diff(fx,x)

In [ ]:
xa = 5*np.pi/8 # value to estimate
xs = np.pi/2 # convenient point

linearApproximation(fx,df,xa,xs) # hint: what field is 'x' in?

In [ ]:
# list of starting values
startVals = np.linspace(-np.pi,2*np.pi,99)
xa = np.mean(startVals)
trueValueXa = fx.subs(x,xa)

# initializations
linapprox = np.zeros(len(startVals))
approxErrors = np.zeros(len(startVals))

# do the approximations and store the results
for i,xs in enumerate(startVals):
  linapprox[i] = linearApproximation(fx,df,xa,xs) # solution to hint in previous cell: set `real=True` when defining symbolic variable `x`
  approxErrors[i] = linapprox[i] - fx.subs(x,xa)


plt.figure(figsize=(8,4))
plt.plot(startVals,approxErrors,'ks',markerfacecolor='w',markersize=9)
plt.axhline(0,linestyle='--',color=[.7,.7,.7],zorder=-3)
plt.axvline(xa,linestyle=':',color=[.8,.8,.8],zorder=-3)
plt.xlabel(r'Initial guess $\left(\zeta_s\right)$')
plt.ylabel('Error from "true" value')

plt.tight_layout()
plt.savefig('diffApps_ex3.png')
plt.show()

# Exercise 9.4: Newton's method

In [ ]:
x = sym.symbols('x',real=True) # force real-valued solutions.

# define the function and its derivative
fx = 2*x**3 - 3
df = sym.diff(fx)

# find the real root
realRoot = sym.solve(fx,x)
realRoot

In [ ]:
# function to implement one iteration
def newtonIter(f,d,x0):
  return x0 - f.subs(x,x0)/d.subs(x,x0)

In [ ]:
x0 = .8 # first guess
x1 = newtonIter(fx,df,x0) # first update
x2 = newtonIter(fx,df,x1) # second update

# print out estimates
print(f'Initial guess:    x = {x0:.3f}')
print(f'First iteration:  x = {x1:.3f}')
print(f'Second iteration: x = {x2:.3f}')
print(f'Analytic root:    x = {realRoot[0].evalf():.3f}')

# Exercise 9.5: Newton's failure

In [ ]:
# 1) uh oh....
newtonIter(fx,df,0)

In [ ]:
# 2) The problem here is that f'(0)=0, which is a failure case of Newton's method.
#    x=0 is *not* a solution to this problem, but the algorithm gets stuck because
#    it cannot move when the gradient is zero
#    (analogously: a marble cannot roll "downhill" on a perfectly flat plane).
#
# 3) You could fix the problem by adding a tiny non-zero offset, but that makes the
#    updated value really huge (e.g., try x0=.0001). Another solution is to start from
#    some random number, although that also risks numerical instability if the initial
#    estimate is too far, which you got a taste of in the previous exercise when x0=-1.

# Exercise 9.6: Newton's method on the example problem

In [ ]:
# define the function and its derivative
fx = 3*x**3 + 2*x - 7*sym.pi
df = sym.diff(fx)

### find the real root

# sym.solve ignores the variable constraints for this problem,
# though you could simply take the third root from the set.
realRoot = sym.solve(fx,x)

# real_roots struggles to find a solution:
# realRoot = sym.real_roots(fx,x)

# solveset works, but you need to extract the arguments from the output, which is a finiteset
# realRoot = sym.solveset(fx,x)#,domain=sym.S.Reals).args[0]
realRoot

In [ ]:
# plot the function
fx_fun = sym.lambdify(x,fx)
xx = np.linspace(-1,3,301)

# plot the function and true root
plt.figure(figsize=(10,5))
plt.plot(xx[[0,-1]],[0,0],'--',linewidth=1,color=[.8,.8,.8])
plt.axvline(sym.N(realRoot),linestyle='--',linewidth=1,color=[.8,.8,.8],label=f'True root (x={sym.N(realRoot):.3f})')
plt.plot(xx,fx_fun(xx),'k',label='f(x)')


# number of iterations
nIters = 4

# initial x0
x0 = 1

# loop over iterations
for i in range(nIters):

  # plot
  x4legend = f'{sym.N(x0):.3f}'
  color = np.ones(3)*(i/(nIters-1))
  plt.plot(x0,fx_fun(x0),'ko',markerfacecolor=color,markersize=np.max([2,12-i*2]),label=r'$x_{%g}=%s$'%(i,x4legend))

  # NOTE: the code below draws tangent lines for each estimate. I thought the figure looked too
  #       crowded and it hurt more than helped. But perhaps you will find it useful. I think it
  #       could look better with some color.
  #tangX = np.array([x0-1,x0+1])
  #tangY = df.subs(x,x0)*(tangX-x0) + fx_fun(x0)
  #plt.plot(tangX,tangY,'--',color=color,linewidth=1,label=f'tangent$_{i}$')

  # update x0
  x0 = newtonIter(fx,df,x0)


plt.legend()
plt.title(r"Newton's method on $f(x)=%s$" %sym.latex(fx),loc='center')
plt.gca().set(xlim=xx[[0,-1]],ylim=[-30,40],xlabel='x',ylabel=r'$y = f(x)$')
plt.tight_layout()
plt.savefig('diffApps_newtonExample.png')
plt.show()

# Exercise 9.7: Newton's convergence

In [ ]:
numIters = 5

# initialize x0 and vector of all guesses
startGuess = 1
xGuess = np.zeros(numIters+1) + startGuess

# loop over iterations
for i in range(1,numIters+1):
  xGuess[i] = newtonIter(fx,df,xGuess[i-1])

# plot the guesses
plt.figure(figsize=(4,5))
plt.plot(xGuess,'s-',color=[.8,.8,.8],linewidth=1,markersize=10,markerfacecolor=[.8,.8,.8],markeredgecolor='k',label='Estimate')
plt.axhline(realRoot,color=[.8,.8,.8],linestyle='--',zorder=-4,label='True root')
plt.xlabel('Iteration')
plt.xticks(range(numIters+1))
plt.ylabel('Root approximation')
plt.legend()

plt.tight_layout()
plt.savefig('diffApps_ex7.png')
plt.show()

# Exercise 9.8: Optimize for cost

The optimization objective is to minimize cost. The constraints are that the shape is rectangular with an area of 400. The price of the fence is distracting irrelevant information; the lengths of the sides are minimized regardless of the exact price.

The tricky part of this problem is seeing that the objective function to minimize is the cost, which is not exactly the same thing as perimeter because Fran covers part of the cost. Draw a diagram to help build the equations below. As with the previous problem, don't worry about making this initial diagram be accurately scaled; the purpose of the diagram is to help us translate the problem stated in words to mathematical equations.

\begin{align}
  A(x,y) &= xy = 400 \\
  P(x,y) &= 2x + 2y \\
  C(x,y) &= 2x+y+y/2
\end{align}

The equation for $P$ is actually not used in the problem. I included it here because it helped me find the equation for cost. $y$ is the side of the fence that Fran shares, and she pays for 1/2 of that.

Next, we use the constraint to transform the problem from two variables into one: $y = 400/x$. And then we can rewrite $C(x,y)$ as a function of $x$ to get $C(x)$. And from there, we can differentiate and find critical points.

\begin{align}
  C(x,y) &= 2x+\frac{3y}{2} \\
  C(x) &= 2x + \frac{3\frac{400}{x}}{2} \\
   &= 2x + \frac{600}{x} \\
   &= 2x + 600x^{-1} \\
   C' &= 2 - 600x^{-2} \\
   0 &= 2 - 600x^{-2} \\
   600x^{-2} &= 2 \\
   x^{-2} &= \frac{1}{300} \\
   x^{2} &= 300 \\
   x &= \sqrt{300} = 10\sqrt{3} \approx 17.32
\end{align}

$x=-10\sqrt{3}$ could mathematically solve the problem, but it makes no sense in the physical world.

So $x=10\sqrt{3}$ is the solution to this problem? Not quite! The problem didn't ask about the length of *one* side; it asked about the lengths of both sides. Fortunately, we have an equation for $y$:
$$ y = 400/x = 400/(10\sqrt{3}) \approx 23.1 $$
So the final answer is that the sides should be around $17.3\times 23.1$ meters, with the longer side shared with Fran.

In these kinds of problems, it's good to "sanity-check" your work by ensuring that the numbers are internally consistent. In this case, we know that the area is 400 m$^2$, and $17.3\times23.1=399.63$. That's not exactly 400, but 17.3 is not exactly $10\sqrt{3}$. Of course, the exact calculation of $xy$ returns the expected 400.


In [ ]:
# formulas
xx = np.linspace(5,70,299)
Cx = 2*xx + 600/xx
dC = 2 - 600/(xx**2)

# plots
_,axs = plt.subplots(2,1,figsize=(10,6))

axs[0].plot(xx,Cx,'k')
axs[0].set(xlim=xx[[0,-1]],ylim=[60,150],xlabel='Length ($x$) of fencing',ylabel='Cost')
axs[0].set_title(r'$\bf{A}$)  $C(x) = 2x + 600x^{-1}$')

axs[1].plot(xx,dC,'k')
axs[1].axhline(0,linestyle='--',color=[.6,.6,.6],zorder=-3)
axs[1].set(xlim=xx[[0,-1]],ylim=[-8,3],xlabel='Length ($x$) of fencing')
axs[1].set_title(r"$\bf{B}$)  $C'(x) = 2 - 600x^{-2}$")

plt.tight_layout()
plt.savefig('diffApps_ex8.png')
plt.show()

# Exercise 9.9: QueBurt's cubes


The objective is to minimize materials cost, and the two constraints are (1) the volume is 200 cm$^3$ and (2) the top and bottom are squares, meaning two of the three sides of the box are equal. The problem statement doesn't list all of the costs, so we will assume that the total surface area of the box is the measure of cost. Let's write down some equations:

\begin{align}
  V(x,y) &= xxy = x^2y = 200 \\
  y &= \frac{200}{x^2} \\
  S(x) &= 2x^2 + 4x\left( \frac{200}{x^2} \right) \\
  &= 2x^2 + \frac{800}{x}
\end{align}

As in previous examples, we used the constraints to reduce the problem from two variables to one. Now we can differentiate the objective function and find its critical points.


\begin{align}
  S' &= 4x - \frac{800}{x^2} \\
  0 &= x - \frac{200}{x^2} \\
  0 &= 1 - \frac{200}{x^3} \\
  \frac{1}{x^3} &= \frac{1}{200} \\
  x^3 &= 200 \\
  x &= \sqrt[3]{200} \approx 5.848 \\
  y &= \frac{200}{\left(\sqrt[3]{200}\right)^2} = (200^{3/3})(200^{-2/3}) \approx 5.848
\end{align}

So, a box with a volume of 200 cm$^3$ has minimal surface area when all sides are equal; hence, a cube.

However, the problem didn't ask for the volume of the box; it asked for the lengths of the edges. So the final answer is that all edges are approximately 5.85 cm.

In [ ]:
# formulas
xx = np.linspace(1,35,299)
Sx = 2*xx**2 + 800/xx
dS = 4*xx - 800/(xx**2)

_,axs = plt.subplots(2,1,figsize=(10,6))

axs[0].plot(xx,Sx,'k')
axs[0].set(xlim=xx[[0,-1]],ylim=[0,2000],xlabel='Length ($x$) of two sides',ylabel='Cost')
axs[0].set_title(r'$\bf{A}$)  $S(x) = 2x^2 + 800x^{-1}$')

axs[1].plot(xx,dS,'k')
axs[1].axhline(0,linestyle='--',color=[.6,.6,.6],zorder=-3)
axs[1].set(xlim=xx[[0,-1]],ylim=[-100,200],xlabel='Length ($x$) of two sides')
axs[1].set_title(r"$\bf{B}$)  $S'(x) = 4x - 800x^{-2}$")

plt.tight_layout()
plt.savefig('diffApps_ex9.png')
plt.show()

# Exercise 9.10: QueBurt's open cubes

The only difference from the previous exercise is in the equation for the total surface area (dropping one side).


\begin{align}
  V &= xxy = x^2y = 200 \\
  y &= \frac{200}{x^2} \\
  S &= x^2 + 4x\left( \frac{200}{x^2} \right) \\
  S' &= 2x - \left( \frac{800}{x^2} \right)
\end{align}

Then we set $S'=0$ and solve for the critical points.

\begin{align}
  x &= \sqrt[3]{400} \approx 7.368 \\
  y &= \frac{200}{\left(\sqrt[3]{400}\right)^2} \approx 3.684
\end{align}

With this added constraint, the boxes shouldn’t be perfect cubes, but instead are wider than tall.

But the solutions for $x$ and $y$ don't actually answer the origin question; the question was about the sides of the faces:

\begin{align}
  x^2 &= \left(\sqrt[3]{400}\right)^2 = 20\sqrt[3]{20} \approx 54.29 \\
  xy &= 200/\sqrt[3]{400} \approx 27.14
\end{align}



In [ ]:
# sympy solution to the derivative problem:
x = sym.symbols('x')
f = x**2 + 4*x * (200/x**2)
sym.solve(sym.diff(f))[0].evalf()

In [ ]:
# formulas
xx = np.linspace(1,35,299)
Sx = xx**2 + 800/xx
dS = 2*xx - 800/(xx**2)

_,axs = plt.subplots(2,1,figsize=(10,6))

axs[0].plot(xx,Sx,'k')
axs[0].set(xlim=xx[[0,-1]],ylim=[0,1200],xlabel='Length ($x$) of two sides',ylabel='Cost')
axs[0].set_title(r'$\bf{A}$)  $S(x) = x^2 + 800x^{-1}$')

axs[1].plot(xx,dS,'k')
axs[1].axhline(0,linestyle='--',color=[.6,.6,.6],zorder=-3)
axs[1].set(xlim=xx[[0,-1]],ylim=[-100,100],xlabel='Length ($x$) of two sides')
axs[1].set_title(r"$\bf{B}$)  $S'(x) = 2x - 800x^{-2}$")

plt.tight_layout()
plt.savefig('diffApps_ex10.png')
plt.show()